In [2]:
import xarray as xr
from pyclim_noresm.aerosol_feedbacks import (partialDF_partialDC,
                                            partialDC_partialDT)
import yaml
import numpy as np

params = snakemake.params
time_slice = params.get('time_slice',slice(5,30))
piControl_slice = params.get('piControl_slice', slice(125,150)) 


In [12]:
def get_equal_lenght_slice(input_files ,time_slice, piControl_slice):
    variables = {}
    len_slice = time_slice.stop-time_slice.start
#     pi_stop = piControl_slice.stop
#     pi_control_slices = {}
    time_dim_lenghts = []
    for key,path in input_files.items():
        if key == 'area_cello':
            area_cello = xr.open_dataset(path)
        else:
            
            temp_ds = xr.open_dataset(str(path))
            if 'year' in temp_ds.dims:
                temp_ds = temp_ds.rename({'year':'time'})
            variables[key]=temp_ds
            time_dim_lenghts.append(len(temp_ds.time))
    
    min_time_lenght = np.array(time_dim_lenghts).min()
    if min_time_lenght > len_slice:
        min_time_lenght = len_slice
    time_slice = slice(time_slice.stop-min_time_lenght, time_slice.stop)
    piControl_slice = slice(piControl_slice.stop-min_time_lenght, piControl_slice.stop)
    for key, data in variables.items():
        
        if key in ['emis_2xCO2', 'emis_PIctrl','t_ctrl', 't_exp']:
#             print(len(data.time),key)
            if len(data.time) <= min_time_lenght:
                variables[key] = data
            elif len(data.time) < piControl_slice.stop:
                print(slice(len(data.time)-min_time_lenght,len(data.time)), data.time, key)
                variables[key] = data.isel(time=slice(len(data.time)-min_time_lenght,len(data.time)))
            else:
                variables[key] = data.isel(time=piControl_slice)
        else:
            variables[key] = data.isel(time=time_slice)
            
    return variables, area_cello

In [13]:
v, area_cello = get_equal_lenght_slice(snakemake.input, time_slice, piControl_slice)

In [14]:
with xr.set_options(keep_attrs=True):
    forcing = v['forcing']
    deltaC_piClim = v['emis_exp'][v['emis_exp'].variable_id]-v['emis_ctrl'][v['emis_ctrl'].variable_id]
    deltaC_piControl = v['emis_2xCO2'][v['emis_2xCO2'].variable_id]-v['emis_PIctrl'][v['emis_PIctrl'].variable_id].assign_coords(time=v['emis_2xCO2'].time)
    deltaT = v['t_exp'][v['t_exp'].variable_id] - v['t_ctrl'][v['t_ctrl'].variable_id].assign_coords(time=v['t_exp'].time)

In [16]:
dFdC, deltaCPiclim, erf= partialDF_partialDC(forcing, deltaC_piClim, area_cello, per_year=False)

In [17]:
dCdT, deltaC_piControl, deltat = partialDC_partialDT(deltaT, deltaC_piControl,area_cello,per_year=False)

In [18]:
data = {}
data['dCdT']={}
data['dCdT']['units'] = 'Tg year-1/K'
data['dCdT']['data'] = (dCdT.values*1e-9).tolist()
data['dFdC']= {}
data['dFdC']['units'] = 'W m-2/tg year-1'
data['dFdC']['data'] = (dFdC.values*1e9).tolist()
data['delta_ems'] = {}
data['delta_ems']['data'] = (deltaCPiclim.values*1e-9).tolist()
data['delta_ems']['units'] = 'Tg year-1'
data['dT_dEms'] = {}
data['dT_dEms']['units'] = 'Tg year-1'
data['dT_dEms']['data'] = (deltaC_piControl.values*1e-9).tolist()
data['alpha'] = {}
data['alpha']['units'] = 'W m-2 K-1'
data['alpha']['data'] = (dCdT.values*dFdC.values).tolist()

In [140]:
with open(snakemake.output.outpath, 'w') as f:
    yaml.safe_dump(data,f,default_flow_style=False)